In [13]:
### IMPORTS ###
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical

## Data Processing

In [35]:
### LOAD AND PREPARE DATA ###

#Load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


#Vectorize images
x_train = np.array([np.float64(x.flatten()) for x in x_train])
x_test = np.array([np.float64(x.flatten()) for x in x_test])


#Normalize images
x_train -= np.mean(x_train, axis = 0)
x_train /= np.std(x_train, axis = 0)
x_test -= np.mean(np.float64(x_test), axis = 0)
x_test /= np.std(x_test, axis = 0)

#One hot encode labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (x_test.shape, y_test.shape))

Train: X=(50000, 3072), y=(50000, 10)
Test: X=(10000, 3072), y=(10000, 10)
